# Classify products to its cluster

In [55]:
# library
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.externals import joblib
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import sqlite3

In [4]:
# make connection to sqlite db
conn = sqlite3.connect('product.db')
c = conn.cursor()

# enable foreign keys
c.execute("PRAGMA foreign_keys = ON")
conn.commit()

In [5]:
# result reproducibility
np.random.seed(42)

In [6]:
# query
# get ranking, reviewcount, salescluster from table 'prodpage'
sqlc = 'SELECT ranking, reviewcount, salescluster FROM prodpage'
c.execute(sqlc)
conn.commit()
product = c.fetchall()
product = pd.DataFrame(product)
product.columns = ['ranking', 'reviewcount', 'salescluster']

In [39]:
product.head()

,ranking,reviewcount,salescluster
0,41580.0,3200,3
1,41233.5,3100,3
2,41571.0,2200,3
3,41052.5,2100,3
4,40821.5,2100,3


In [36]:
product['salescluster'] = product['salescluster'].astype('int')

In [38]:
# Total number of cluster 1.0
len(product[product['salescluster'] == 1])/len(product)

0.4768006539090297

In [40]:
# Total number of cluster 2.0
len(product[product['salescluster'] == 2])/len(product)

0.5070199057601692

In [41]:
# Total number of cluster 1.0
len(product[product['salescluster'] == 3])/len(product)

0.01617944033080104

In [42]:
X = product[['ranking', 'reviewcount']].values
y = product['salescluster'].values

In [43]:
# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [44]:
# train-test set split
# Use stratified sampling, because classes are skewed
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2,
                                                   random_state=42,
                                                   shuffle=True,
                                                   stratify=y)

In [45]:
len(y_train[y_train == 1.])/len(y_train), len(y_train[y_train == 2.])/len(y_train), len(y_train[y_train == 3.])/len(y_train)

(0.4768000961654045, 0.5070320952037505, 0.016167808630845052)

In [46]:
len(y_test[y_test == 1.])/len(y_test), len(y_test[y_test == 2.])/len(y_test), len(y_test[y_test == 3.])/len(y_test)

(0.4768028846153846, 0.5069711538461539, 0.01622596153846154)

In [ ]:
# Skewed Classes
# https://florianhartl.com/thoughts-on-machine-learning-dealing-with-skewed-classes.html
# https://news.ycombinator.com/item?id=4440560

## 0. Dumb Classifier

In [60]:
from sklearn.base import BaseEstimator

In [61]:
class DumbClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)
dumb = DumbClassifier()
cross_val_score(dumb, X_train, y_train, cv=5, scoring='accuracy')

array([0., 0., 0., 0., 0.])

## 1. Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression

In [47]:
logreg = LogisticRegression(random_state=42, multi_class='multinomial',
                           class_weight={3: 3.,
                                        2: 1.5,
                                        1: 1.5})

In [49]:
# Grid Search
parameters = [{
    'C': [1.0, 10.0, 15.0, 20.0, 30.0, 40.0,
         50.0],
    'solver': ['newton-cg', 'saga',
              'lbfgs'],
    'penalty': ['l2']
}]

grid_search = GridSearchCV(estimator=logreg,
                          param_grid=parameters,
                          scoring=['accuracy', 'f1_macro',
                                  'f1_micro'],
                          cv=5,
                          verbose=2,
                          refit='f1_micro',
                          n_jobs=-1)

In [50]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV] C=1.0, penalty=l2, solver=newton-cg .............................
[CV] C=1.0, penalty=l2, solver=newton-cg .............................
[CV] C=1.0, penalty=l2, solver=newton-cg .............................
[CV] C=1.0, penalty=l2, solver=newton-cg .............................
[CV] C=1.0, penalty=l2, solver=newton-cg .............................
[CV] C=1.0, penalty=l2, solver=saga ..................................
[CV] C=1.0, penalty=l2, solver=saga ..................................
[CV] C=1.0, penalty=l2, solver=saga ..................................
[CV] C=1.0, penalty=l2, solver=saga ..................................
[CV] C=1.0, penalty=l2, solver=saga ..................................
[CV] C=1.0, penalty=l2, solver=lbfgs .................................
[CV] C=1.0, penalty=l2, solver=lbfgs .................................
[CV] .............. C=1.0, penalty=l2, solver=newton-cg, total=   0.5s
[CV] C=1.0, pen

/usr/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ................... C=1.0, penalty=l2, solver=saga, total=   1.7s
[CV] C=10.0, penalty=l2, solver=lbfgs ................................


/usr/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ................... C=1.0, penalty=l2, solver=saga, total=   1.8s
[CV] ................. C=10.0, penalty=l2, solver=lbfgs, total=   0.4s


...

[CV] .................. C=50.0, penalty=l2, solver=saga, total=   1.1s
[CV] .................. C=50.0, penalty=l2, solver=saga, total=   1.2s


/usr/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] .................. C=50.0, penalty=l2, solver=saga, total=   1.3s


[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:    9.2s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight={3: 3.0, 2: 1.5, 1: 1.5}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=42, solver='liblinear', tol=0.0001, verbose=0,
          warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [1.0, 10.0, 15.0, 20.0, 30.0, 40.0, 50.0], 'solver': ['newton-cg', 'saga', 'lbfgs'], 'penalty': ['l2']}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn',
       scoring=['accuracy', 'f1_macro', 'f1_micro'], verbose=2)

In [51]:
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

In [52]:
best_parameters

{'C': 40.0, 'penalty': 'l2', 'solver': 'newton-cg'}

In [53]:
grid_search.best_score_

0.9997896381776655

In [56]:
# confusion matrix
y_train_pred = cross_val_predict(best_estimator, X_train, y_train, cv=5,
                                verbose=2, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


In [57]:
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

array([[15864,     0,     2],
       [    4, 16868,     0],
       [    1,     0,   537]])

In [58]:
# confusion matrix
y_test_pred = cross_val_predict(best_estimator, X_test, y_test, cv=5,
                                verbose=2, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [59]:
conf_mx_test = confusion_matrix(y_test, y_test_pred)
conf_mx_test

array([[3965,    1,    1],
       [   5, 4213,    0],
       [   0,    0,  135]])

In [62]:
# f1 score
print(f1_score(y_train, y_train_pred, average='micro'))
print(f1_score(y_test, y_test_pred, average='micro'))

0.9997896381776655
0.9991586538461539


In [67]:
# Save logreg model
joblib.dump(best_estimator, './training/grupClass_logreg.pkl')

['./training/grupClass_logreg.pkl']

## 2. Decision Tree

In [64]:
from sklearn.tree import DecisionTreeClassifier

In [65]:
destree = DecisionTreeClassifier(random_state=42,
                                class_weight={3: 3.,
                                              2: 1.5,
                                              1: 1.5}
                                )

In [76]:
# Grid Search
parameters2 = [{
    'max_depth': [3, 4, 5, 7, 9, 10],
    'min_samples_leaf': [4, 5, 6],
    'min_samples_split': [5, 6, 7, 8, 9, 10, 11, 12],
    'max_features': ['auto', 'log2']
}]

grid_search2 = GridSearchCV(estimator=destree,
                          param_grid=parameters2,
                          scoring=['accuracy', 'f1_macro',
                                  'f1_micro'],
                          cv=5,
                          verbose=2,
                          refit='f1_micro',
                          n_jobs=-1)

In [77]:
grid_search2.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples

[CV] max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=7 
[CV] max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=12 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=11 
[CV] max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=6 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=12, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=9, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=12 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, mi

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s


[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=11, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=5 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=12, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV] max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=5 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=9, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=11 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV] max_depth=3, max_features

[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=12, total=   0.0s
[CV] max_depth=3, max_features=log2, min_samples_leaf=4, min_samples_split=9 
[CV] max_depth=3, max_features=log2, min_samples_leaf=4, min_samples_split=6 
[CV] max_depth=3, max_features=log2, min_samples_leaf=4, min_samples_split=11 
[CV]  max_depth=3, max_features=log2, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV] max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=9 
[CV] max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=8 
[CV] max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=12 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=8, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV]  max_depth=3, max_features=auto, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV]  max_depth=3, max_features=log2, min_samples_leaf=4, min_

...

[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=7, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=11 
[CV] max_depth=10, max_features=auto, min_samples_leaf=6, min_samples_split=11 
[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV]  max_depth=10, max_features=auto, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=10, max_features=auto, min_samples_leaf=6, min_samples_split=9 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=8, total=   0.0s
[CV] max_depth=10, max_features=auto, min_samples_leaf=6, min_samples_split=12 
[CV] max_depth=10, max_features=auto, min_samples_leaf=6, min_samples_split=8 
[CV]  max_depth=10, max_features=auto, min_samples_leaf=6, min_sampl

[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=7 
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=9 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=11, total=   0.0s
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=9, total=   0.0s
[CV]  max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=6, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=11 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV]  max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=12, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=7 
[CV] max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=9 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=5, total=   0.0s
[CV] max_depth=10, max_features=log2,

[CV] max_depth=10, max_features=log2, min_samples_leaf=6, min_samples_split=11 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=8, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=8 
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=11 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=6, min_samples_split=10 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=9, total=   0.0s
[CV]  max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=12, total=   0.0s
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=10 
[CV] max_depth=10, max_features=log2, min_samples_leaf=5, min_samples_split=12 
[CV]  max_depth=10, max_features=log2, min_samples_leaf=6, min_samples_split=9, total=   0.0s
[CV]  max_depth=10, max_features=log2, min_sample

[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:    7.4s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight={3: 3.0, 2: 1.5, 1: 1.5},
            criterion='gini', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'max_depth': [3, 4, 5, 7, 9, 10], 'min_samples_leaf': [4, 5, 6], 'min_samples_split': [5, 6, 7, 8, 9, 10, 11, 12], 'max_features': ['auto', 'log2']}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn',
       scoring=['accuracy', 'f1_macro', 'f1_micro'], verbose=2)

In [78]:
best_parameters2 = grid_search2.best_params_
best_estimator2 = grid_search2.best_estimator_

In [79]:
best_parameters2

{'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 9}

In [80]:
grid_search2.best_score_

0.9991585527106623

In [81]:
# confusion matrix
y_train_pred2 = cross_val_predict(best_estimator2, X_train, y_train, cv=5,
                                verbose=2, n_jobs=-1)

conf_mx2 = confusion_matrix(y_train, y_train_pred2)
conf_mx2

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


array([[15855,    10,     1],
       [   13, 16859,     0],
       [    4,     0,   534]])

In [82]:
# confusion matrix
y_test_pred2 = cross_val_predict(best_estimator2, X_test, y_test, cv=5,
                                verbose=2, n_jobs=-1)

conf_mx_test2 = confusion_matrix(y_test, y_test_pred2)
conf_mx_test2

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


array([[3951,   12,    4],
       [  14, 4204,    0],
       [   2,    0,  133]])

In [83]:
# f1 score
print(f1_score(y_train, y_train_pred2, average='micro'))
print(f1_score(y_test, y_test_pred2, average='micro'))

0.9991585527106623
0.9961538461538462


In [84]:
# Save destree model
joblib.dump(best_estimator2, './training/grupClass_destree.pkl')

['./training/grupClass_destree.pkl']

## 3. C-Support Vector Classification

In [85]:
from sklearn.svm import SVC

In [86]:
suvec = SVC(random_state=42, 
           decision_function_shape='ovo',
           class_weight={3: 3.,
                         2: 1.5,
                         1: 1.5})

In [89]:
suvec.get_params().keys()

dict_keys(['C', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [90]:
# Grid Search
parameters3 = [{
    'C': [0.001, 0.01, 0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'sigmoid']
}]

grid_search3 = GridSearchCV(estimator=suvec,
                          param_grid=parameters3,
                          scoring=['accuracy', 'f1_macro',
                                  'f1_micro'],
                          cv=5,
                          verbose=2,
                          refit='f1_micro',
                          n_jobs=-1)

In [91]:
grid_search3.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] C=0.001, gamma=0.001, kernel=poly ...............................
[CV] C=0.001, gamma=0.001, kernel=poly ...............................
[CV] C=0.001, gamma=0.001, kernel=sigmoid ............................
[CV] C=0.001, gamma=0.001, kernel=poly ...............................
[CV] C=0.001, gamma=0.001, kernel=sigmoid ............................
[CV] C=0.001, gamma=0.001, kernel=poly ...............................
[CV] C=0.001, gamma=0.001, kernel=poly ...............................


/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-s

[CV] ................ C=0.001, gamma=0.001, kernel=poly, total=  21.6s
[CV] C=0.001, gamma=0.001, kernel=sigmoid ............................
[CV] ................ C=0.001, gamma=0.001, kernel=poly, total=  21.6s
[CV] C=0.001, gamma=0.001, kernel=sigmoid ............................
[CV] ................ C=0.001, gamma=0.001, kernel=poly, total=  21.8s
[CV] C=0.001, gamma=0.001, kernel=sigmoid ............................
[CV] ................ C=0.001, gamma=0.001, kernel=poly, total=  22.3s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ................ C=0.001, gamma=0.001, kernel=poly, total=  22.3s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-s

[CV] ............. C=0.001, gamma=0.001, kernel=sigmoid, total=  49.3s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ............. C=0.001, gamma=0.001, kernel=sigmoid, total=  50.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ................. C=0.001, gamma=0.001, kernel=rbf, total=  50.7s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ................. C=0.001, gamma=0.001, kernel=rbf, total=  51.1s
[CV] C=0.001, gamma=0.01, kernel=poly ................................
[CV] ................. C=0.001, gamma=0.001, kernel=rbf, total=  50.7s
[CV] C=0.001, gamma=0.01, kernel=poly ................................
[CV] ................. C=0.001, gamma=0.001, kernel=rbf, total=  50.8s
[CV] C=0.001, gamma=0.01, kernel=poly ................................
[CV] ................. C=0.001, gamma=0.001, kernel=rbf, total=  51.2s
[CV] C=0.001, gamma=0.01, kernel=poly ................................


...

[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   2.4s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   2.3s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   6.3s
[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   6.2s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] C=10, gamma=0.01, kernel=poly ...................................


/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   6.5s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   6.7s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   6.3s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   2.3s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] ................. C=10, gamma=0.01, kernel=sigmoid, total=   3.0s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] ................. C=10, gamma=0.01, kernel=sigmoid, total=   2.8s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] ................. C=10, gamma=0.01, kernel=sigmoid, total=   3.0s
[CV] C=10, gamma=0.01, kernel=sigmoid ................................
[CV] .

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight={3: 3.0, 2: 1.5, 1: 1.5}, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['rbf', 'poly', 'sigmoid']}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn',
       scoring=['accuracy', 'f1_macro', 'f1_micro'], verbose=2)

In [92]:
best_parameters3 = grid_search3.best_params_
best_estimator3 = grid_search3.best_estimator_

In [93]:
best_parameters3

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [94]:
grid_search3.best_score_

0.9994590695997115

In [95]:
# confusion matrix
y_train_pred3 = cross_val_predict(best_estimator3, X_train, y_train, cv=5,
                                verbose=2, n_jobs=-1)


[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


In [96]:
conf_mx3 = confusion_matrix(y_train, y_train_pred3)
conf_mx3

array([[15864,     0,     2],
       [   16, 16856,     0],
       [    0,     0,   538]])

In [97]:
# confusion matrix
y_test_pred3 = cross_val_predict(best_estimator3, X_test, y_test, cv=5,
                                verbose=2, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [98]:
conf_mx_test3 = confusion_matrix(y_test, y_test_pred3)
conf_mx_test3

array([[3966,    1,    0],
       [   8, 4210,    0],
       [   0,    0,  135]])

In [99]:
# f1 score
print(f1_score(y_train, y_train_pred3, average='micro'))
print(f1_score(y_test, y_test_pred3, average='micro'))

0.9994590695997115
0.9989182692307692


In [100]:
# Save logreg model
joblib.dump(best_estimator3, './training/grupClass_svc.pkl')

['./training/grupClass_svc.pkl']